### https://github.com/nlpai-lab/KULLM
merge_lora.py 참고함.

finetune_kullm_polyglot.py을 통해 생성된 adapter_model.bin 파일을 불러와서 모델 답변 확인해보기 <br>

LoRA의 방식으로 파인튜닝하면 사전 학습 모델의 파라미터는 고정되기 때문에 사전학습 모델의 파라미터와 파인튜닝해서 얻은 adapter_model.bin 파라미터를 합치는 과정이 필요하다.

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training
)
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast
import sys


def answer_chat(input,model,tokenizer,device=1):
    inputs = tokenizer(input, return_tensors="pt", return_token_type_ids=False).to(device)

    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**inputs, max_new_tokens=300, pad_token_id=2)[0], skip_special_tokens=True)) # max_new_tokens 너무 크면 같은 단어의 반복
        


데이터 컬럼 : ['instruction', 'input', 'output']

In [15]:
from datasets import load_dataset
data_path = 'data.jsonl'
data = load_dataset("json", data_files=data_path)
data

Found cached dataset json (/home/brianjang7/.cache/huggingface/datasets/json/default-a76f94fd1b103ee7/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 90309
    })
})

In [ ]:
pd.DataFrame(data)

## merge 방법 1

사용량이 적은 GPU 자동할당

In [2]:
# Set the available GPU devices
# available_gpus = [0]  # 사용 가능한 GPU가 1개일 경우
available_gpus = [0, 1]  # 사용 가능한 GPU가 2개일 경우

# Function to find the GPU with the least memory usage
def get_least_memory_gpu():
    least_memory = float('inf')
    least_memory_gpu = None
    for gpu in available_gpus:
        allocated_memory = torch.cuda.memory_allocated(gpu)
        if allocated_memory < least_memory:
            least_memory = allocated_memory
            least_memory_gpu = gpu
    return least_memory_gpu

# Allocate the remaining GPU memory to the current process
device = get_least_memory_gpu()
torch.cuda.set_device(device)
print('device:',device)

device: 0


Lora 적용하기

In [3]:
def apply_lora(base_model_path, lora_path):
    print(f"Loading the base model from {base_model_path}")
    base = AutoModelForCausalLM.from_pretrained(base_model_path, torch_dtype=torch.float16, low_cpu_mem_usage=True,device_map=device)
    base_tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    base_tokenizer.pad_token_id = 0
    
    print(f"Loading the LoRA adapter from {lora_path}")

    lora_model = PeftModel.from_pretrained(
        base,
        lora_path,
        torch_dtype=torch.float16,
    )

    print("Applying the LoRA")
    merged_model = lora_model.merge_and_unload()

    return merged_model, base_tokenizer



In [ ]:
for i in range(3):
    print(df['output'][i])

## Prompt template

"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n{instruction}\n\n### 입력:\n{input}\n\n### 응답:\n"

In [ ]:
model_id = 'nlpai-lab/kullm-polyglot-5.8b-v2'
lora_path = './polyglot-ko-5.8B_KULLM_patent_30000'
model, tokenizer = apply_lora(model_id, lora_path)

In [ ]:
for i in range(3):
    instruction = df['instruction'][i]
    input = df['input'][i]
    
    query = f"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n{instruction}\n\n### 입력:\n{input}\n\n### 응답:\n"
    answer_chat(query,model=model,tokenizer=tokenizer,device=device)
    print()

## merge 방법 2

메모리 사용량이 적은 GPU로 device 설정

In [6]:
# Set the available GPU devices
available_gpus = [0, 1]  # Update this list with the available GPU device IDs

# Function to find the GPU with the least memory usage
def get_least_memory_gpu():
    least_memory = float('inf')
    least_memory_gpu = None
    for gpu in available_gpus:
        allocated_memory = torch.cuda.memory_allocated(gpu)
        if allocated_memory < least_memory:
            least_memory = allocated_memory
            least_memory_gpu = gpu
    return least_memory_gpu

# Allocate the remaining GPU memory to the current process
device = get_least_memory_gpu()
torch.cuda.set_device(device)
print('device:',device)

device: 0


In [3]:
model_id = 'nlpai-lab/kullm-polyglot-5.8b-v2'

Lora config

In [4]:
lora_r= 16           # 153 k 데이터 8 에폭 학습할 때
lora_alpha = 8
lora_dropout= 0.05
lora_target_modules = ["query_key_value", "xxx"]

load pretrained model weight and lora weight

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map=device,
)

old_state_dict = model.state_dict
model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
    model, type(model)
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
    
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)


checkpoint_name =  "output/checkpoint-50000/adapter_model.bin"
adapters_weights = torch.load(checkpoint_name)
set_peft_model_state_dict(model, adapters_weights)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

_IncompatibleKeys(missing_keys=['base_model.model.gpt_neox.embed_in.weight', 'base_model.model.gpt_neox.layers.0.input_layernorm.weight', 'base_model.model.gpt_neox.layers.0.input_layernorm.bias', 'base_model.model.gpt_neox.layers.0.post_attention_layernorm.weight', 'base_model.model.gpt_neox.layers.0.post_attention_layernorm.bias', 'base_model.model.gpt_neox.layers.0.attention.rotary_emb.inv_freq', 'base_model.model.gpt_neox.layers.0.attention.query_key_value.weight', 'base_model.model.gpt_neox.layers.0.attention.query_key_value.bias', 'base_model.model.gpt_neox.layers.0.attention.query_key_value.lora_A.default.weight', 'base_model.model.gpt_neox.layers.0.attention.query_key_value.lora_B.default.weight', 'base_model.model.gpt_neox.layers.0.attention.dense.weight', 'base_model.model.gpt_neox.layers.0.attention.dense.bias', 'base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.weight', 'base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.bias', 'base_model.model.gpt_neox.layers.0.mlp.d

학습 데이터 inference

In [ ]:
for i in range(3):
    instruction = df['instruction'][i]
    input = df['input'][i]
    
    query = f"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n{instruction}\n\n### 입력:\n{input}\n\n### 응답:\n"
    answer_chat(query,model=model,tokenizer=tokenizer,device=device)
    print()

instruction에 질문 날리기

In [13]:
instruction = '가방이 무거워? 책이 무거워? 이유를 설명해줘.'
input = ''      # 예시가 없으면 input은 공란

query = f"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n{instruction}\n\n### 입력:\n{input}\n\n### 응답:\n"
answer_chat(query,model=model,tokenizer=tokenizer,device=device)
print()

/home/brianjang7/home1/.venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
가방이 무거워? 책이 무거워? 이유를 설명해줘.

### 입력:


### 응답:
죄송하지만 가방과 책의 무게를 비교할 수 있는 구체적인 데이터나 정보가 없습니다. 가방과 책의 무게는 개인의 체격, 체격, 가방의 크기와 모양에 따라 달라질 수 있습니다.

